## Homework_6.1 

- Submitted by Ruchita Alate

### Loading Necessary Libraries

In [1]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Reading the dataset

In [2]:
# Reading the dataset
licenses_url='https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/licenses_fall2022.csv'
licenses = pd.read_csv(licenses_url)
licenses.head(5)

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,None,None,None,N,03/18/2022,None,None,None,None,None
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,None,None,None,N,08/16/2006,None,None,None,None,None
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,None,None,None,N,05/26/2006,None,None,None,None,None
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,None,None,None,N,11/12/2021,None,None,None,None,None
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,None,None,None,N,05/30/2006,None,None,None,None,None


### Data Pre-processing

In [3]:
# Checking the count for each State
licenses['State'].value_counts()

IL    9026
IN     139
WI      94
MO      91
FL      86
CA      84
TX      62
IA      55
MI      30
AZ      29
GA      27
OH      21
CO      20
MN      19
NY      19
        16
KY      16
AR      14
NC      14
VA      14
UT      13
TN      11
NV      10
MS      10
PA      10
WA       9
MD       9
AL       8
SC       7
NJ       5
KS       5
MA       4
OR       3
MT       3
LA       3
HI       3
OK       2
NM       2
BC       1
ID       1
NH       1
NE       1
ND       1
VT       1
WY       1
Name: State, dtype: int64

In [4]:
# Filtering the states with 10 or more results and keep it in the 'licenses' DataFrame
licenses = licenses.groupby('State').filter(lambda x: len(x) >= 10)

# Verifying the filtered data
print(licenses['State'].value_counts())

IL    9026
IN     139
WI      94
MO      91
FL      86
CA      84
TX      62
IA      55
MI      30
AZ      29
GA      27
OH      21
CO      20
MN      19
NY      19
        16
KY      16
NC      14
VA      14
AR      14
UT      13
TN      11
MS      10
NV      10
PA      10
Name: State, dtype: int64


The states with fewer than 10 records were removed, as they contained insufficient data, which did not add value and skewed the visualizations. This filtering allows for a more meaningful distribution of license statuses, focusing on states with a broader dataset to enhance the accuracy and clarity of the visual representation.

In [5]:
# Checking the dataset columns and types
licenses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9930 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   _id                                      9930 non-null   int64 
 1   License Type                             9930 non-null   object
 2   Description                              9930 non-null   object
 3   License Number                           9870 non-null   object
 4   License Status                           9930 non-null   object
 5   Business                                 9930 non-null   object
 6   Title                                    109 non-null    object
 7   First Name                               9537 non-null   object
 8   Middle                                   3602 non-null   object
 9   Last Name                                9537 non-null   object
 10  Prefix                                   3 non-null      obj

In [6]:
# Checking Data Dimensionality
licenses.shape

(9930, 31)

In [7]:
#Checking for missing values in the dataset
missing_values = licenses.isnull().sum()
print(missing_values)

_id                                           0
License Type                                  0
Description                                   0
License Number                               60
License Status                                0
Business                                      0
Title                                      9821
First Name                                  393
Middle                                     6328
Last Name                                   393
Prefix                                     9927
Suffix                                     9527
Business Name                                 0
BusinessDBA                                9817
Original Issue Date                           0
Effective Date                              777
Expiration Date                             491
City                                         11
State                                         0
Zip                                          71
County                                  

In [8]:
#Checking for percentage missing values in the dataset
missing_percentage = (licenses.isnull().sum() / len(licenses)) * 100
print(missing_percentage)

_id                                         0.000000
License Type                                0.000000
Description                                 0.000000
License Number                              0.604230
License Status                              0.000000
Business                                    0.000000
Title                                      98.902316
First Name                                  3.957704
Middle                                     63.726083
Last Name                                   3.957704
Prefix                                     99.969789
Suffix                                     95.941591
Business Name                               0.000000
BusinessDBA                                98.862034
Original Issue Date                         0.000000
Effective Date                              7.824773
Expiration Date                             4.944612
City                                        0.110775
State                                       0.

In [9]:
# Dropping the columns with more than 90% missing values: 
licenses.drop(columns=['Title', 'Prefix', 'Suffix', 'BusinessDBA'], inplace=True)

In [10]:
licenses['Middle'].fillna('N/A', inplace=True) 

For the missing values 'Middle' ( Middle Name) - I have filled the column with "N/A". This is done to maintain data integrity while acknowledging the absence of middle names. This ensures consistency without introducing incorrect information, and the middle name can serve as an important identifier in cases where individuals share the same first and last name.

In [11]:
# Dropping rows with missing values when the percentage of missing data is below 10%
licenses.dropna(subset=['License Number', 'First Name', 'Last Name', 'Effective Date','Expiration Date', 'City', 'Zip','County','Specialty/Qualifier', 'Controlled Substance Schedule','Action'], inplace=True)

In [12]:
# veryfing that there are no missing values in the dataset
missing_values = licenses.isnull().sum()
print(missing_values)

_id                                        0
License Type                               0
Description                                0
License Number                             0
License Status                             0
Business                                   0
First Name                                 0
Middle                                     0
Last Name                                  0
Business Name                              0
Original Issue Date                        0
Effective Date                             0
Expiration Date                            0
City                                       0
State                                      0
Zip                                        0
County                                     0
Specialty/Qualifier                        0
Controlled Substance Schedule              0
Delegated Controlled Substance Schedule    0
Ever Disciplined                           0
LastModifiedDate                           0
Case Numbe

In [13]:
# Checking Data Dimensionality after handling missing values
licenses.shape

(8051, 27)

In [14]:
# Checking the data types of each column
licenses.dtypes

_id                                         int64
License Type                               object
Description                                object
License Number                             object
License Status                             object
Business                                   object
First Name                                 object
Middle                                     object
Last Name                                  object
Business Name                              object
Original Issue Date                        object
Effective Date                             object
Expiration Date                            object
City                                       object
State                                      object
Zip                                        object
County                                     object
Specialty/Qualifier                        object
Controlled Substance Schedule              object
Delegated Controlled Substance Schedule    object


In [15]:
# Converting date columns to datetime
licenses['Original Issue Date'] = pd.to_datetime(licenses['Original Issue Date'], errors='coerce', format='%m/%d/%Y')
licenses['Effective Date'] = pd.to_datetime(licenses['Effective Date'], errors='coerce', format='%m/%d/%Y')
licenses['Expiration Date'] = pd.to_datetime(licenses['Expiration Date'], errors='coerce' , format='%m/%d/%Y')
licenses['LastModifiedDate'] = pd.to_datetime(licenses['LastModifiedDate'], errors='coerce', format='%m/%d/%Y')
# Creating New Year Columns for above dates 
licenses['Original Issue Year']= licenses['Original Issue Date'].dt.year
licenses['Effective Date Year']= licenses['Effective Date'].dt.year
licenses['Expiration Date Year']= licenses['Expiration Date'].dt.year
licenses['Last Modified Date Year']= licenses['LastModifiedDate'].dt.year


In [16]:
# Converting object columns to category
for col in licenses.select_dtypes(include='object').columns:
    licenses[col] = licenses[col].astype('category')
# Checking the data types of each column
licenses.dtypes

_id                                                 int64
License Type                                     category
Description                                      category
License Number                                   category
License Status                                   category
Business                                         category
First Name                                       category
Middle                                           category
Last Name                                        category
Business Name                                    category
Original Issue Date                        datetime64[ns]
Effective Date                             datetime64[ns]
Expiration Date                            datetime64[ns]
City                                             category
State                                            category
Zip                                              category
County                                           category
Specialty/Qual

In this preprocessing workflow, the dataset is first read from a URL. The states with fewer than 10 records were removed, as they contained insufficient data and then columns with more than 90% missing values are dropped. Missing values in the 'Middle' column are filled with "N/A" to maintain data integrity. Rows with missing critical data, such as 'License Number', 'First Name', 'Last Name', 'Effective Date', and others, are dropped. After handling missing values, the dataset's dimensionality is updated, and date columns (such as 'Original Issue Date', 'Effective Date', 'Expiration Date', and 'LastModifiedDate') are converted to the datetime format. New year columns are created from these dates. Finally, all object-type columns are converted to the 'category' data type to optimize memory usage and performance. The preprocessed data is then used for the visulaisations below. 

## Visualisations

In [17]:
# Visualization 1: License Types by State 
# Bar graph is used

bar_chart = alt.Chart(licenses).mark_bar().encode(
    x=alt.X('count(id):Q', title='Number of Licenses'),
    y=alt.Y('License Type:N', sort='-x', title='License Type'),
    color='License Type:N',
    tooltip=['License Type:N', 'count(id):Q']
).properties(
    title='License Types by State',
    width=800,
    height=400  
).add_selection(
    state_selection := alt.selection_single(fields=['State'], bind=alt.binding_select(options=licenses['State'].unique().tolist(), name='Select State'))
).transform_filter(
    state_selection
)

bar_chart
bar_chart.save('License Types by State.html')


/var/folders/9k/fp5ky6215vdb5ltptvrg3rk40000gn/T/ipykernel_423/463223680.py:14: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  state_selection := alt.selection_single(fields=['State'], bind=alt.binding_select(options=licenses['State'].unique().tolist(), name='Select State'))
/var/folders/9k/fp5ky6215vdb5ltptvrg3rk40000gn/T/ipykernel_423/463223680.py:13: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


### **1. Description**  
This bar chart visualizes the distribution of license types across different states. It displays the number of licenses for each type, where the height of each bar represents the frequency of a particular license type within the selected state. By providing a breakdown of license types, the chart helps to understand the prevalence of different types of licenses across states, offering insight into which license types are more common in the chosen state.

### **2. Encoding Types**  
- **X (Quantitative)**: The `count(id)` variable is used to encode the number of licenses for each license type. The height of each bar represents the number of licenses for that specific type.
- **Y (Ordinal)**: The `License Type` variable is encoded on the y-axis, sorted by the number of licenses in descending order to highlight the most frequent license types at the top.
- **Color**: Categorical encoding (`License Type`) is used to color the bars by license type, visually distinguishing between the different license types.
- **Tooltip**: Displays detailed information on the `License Type` and the corresponding count of licenses when hovering over each bar.

### **3. Color Scheme**  
A categorical color scheme is applied to differentiate between the different license types. This color differentiation enhances clarity, making it easier for viewers to distinguish between each type of license. The colors are automatically assigned by Altair, ensuring consistent and clear visual communication.

### **4. Data Transformations**  
The data is filtered based on the selected state, allowing users to view the license type distribution for a specific state. The data is aggregated by `License Type` and the count of licenses (`count(id)`) is calculated to represent the number of licenses for each type. This transformation helps to focus on the specific license type distribution within the state, ensuring that the data is aggregated and presented accurately.

### **5. Interactivity**  
The bar chart includes an interactive dropdown menu (`state_selection`) that allows users to select a state. This interactivity enables users to dynamically filter the data and examine the license type distribution for any specific state. The dropdown allows for a smooth and flexible exploration of the data.

### **6. Impact of Interactivity**  
The interactivity of the bar chart enhances its usefulness by allowing users to filter the data based on the selected state. By choosing different states, users can easily compare the license type distribution across various regions. This interaction makes the visualization more personalized and relevant to the user's specific interests, improving data exploration and insight extraction. It also allows for a clearer focus on the license types that are most common in each state, providing a better understanding of regional differences in license distributions.


In [18]:
# Visualisation 2 : Top 5 License Status Distribution by Cities

filtered_data = licenses.groupby('City').filter(lambda x: x['License Status'].nunique() >= 5)

pie_chart = alt.Chart(filtered_data).mark_arc().encode(
    theta=alt.Theta('count_id:Q', title='Number of Licenses'),
    color=alt.Color('License Status:N', title='License Status'),
    tooltip=['License Status:N', 'count_id:Q']
).properties(
    title='Top 5 License Status Distribution by Cities'
).add_selection(
    city_selection := alt.selection_single(
        fields=['City'],
        bind=alt.binding_radio(
            options=filtered_data['City'].unique().tolist(),
            name='Select City'
        )
    )
).transform_filter(
    city_selection
).transform_aggregate(
    count_id='count(id)',
    groupby=['City', 'License Status']
).transform_window(
    rank='rank()',
    sort=[
        alt.SortField('count_id', order='descending'),
        alt.SortField('License Status', order='ascending')  
    ]
).transform_filter(
    alt.datum.rank <= 5
)

pie_chart
pie_chart.save('Top 5 License Status Distribution by Cities.html')

/var/folders/9k/fp5ky6215vdb5ltptvrg3rk40000gn/T/ipykernel_423/2105283055.py:12: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  city_selection := alt.selection_single(
/var/folders/9k/fp5ky6215vdb5ltptvrg3rk40000gn/T/ipykernel_423/2105283055.py:11: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


### **1. Description**  
This pie chart visualizes the distribution of the top 5 most frequent license statuses within cities that have at least 5 distinct license types. The chart shows the proportion of each license status within the selected city, with the size of each segment proportional to the number of licenses with that particular status. By focusing on cities with a minimum of 5 license types, this visualization allows users to explore the most common license statuses in more diverse cities with a richer variety of license types.

### **2. Encoding Types**  
- **Theta (Angle)**: Quantitative encoding (`count_id`) represents the number of licenses with each status, determining the size of the pie segments.
- **Color**: Categorical encoding (`License Status`) distinguishes between different license statuses, helping to visually separate the statuses for easier interpretation.
- **Tooltip**: Displays detailed information on each license status and the associated count when hovering over the segments.

### **3. Color Scheme**  
A categorical color scheme is applied to differentiate the license statuses. This ensures that each license status is clearly distinguishable from others, enhancing the visual clarity of the chart. The colors are automatically assigned by Altair, ensuring consistent design and easy differentiation between the statuses.

### **4. Data Transformations**  
The data is filtered to include only those cities that have at least 5 distinct license types (`filtered_data`). It is then aggregated by `License Status` and `City`, with the `count(id)` computed to show the total number of licenses for each status within the city. The data is then ranked by the count of licenses in descending order, and only the top 5 license statuses for each city are displayed. This transformation allows the focus to be on the most significant license statuses while filtering out cities with fewer types of licenses.

### **5. Interactivity**  
The pie chart includes an interactive radio button feature (`city_selection`), enabling users to select a city from the list. This allows users to explore the top 5 license statuses for different cities, making the data exploration more dynamic and interactive.

### **6. Impact of Interactivity**  
Interactivity enhances the usability of the pie chart by allowing users to select a city and view its top 5 license status distribution. This gives users a tailored, more in-depth view of the data, enabling them to focus on specific cities of interest. The radio button selection makes it easier to choose cities from the list without overwhelming the user with too many options at once. Overall, this interaction improves the accessibility of the data and makes the visualization more engaging and customizable.